In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**List Library**

In [ ]:
# Importing library
import numpy as np
import pandas as pd

# BeautifulSoup = hapus tag html
from bs4 import BeautifulSoup 
import re # regular expressions (regex)

# natural language tool kits
from nltk.corpus import stopwords
import nltk

# word2vec library
from gensim.models import word2vec
import itertools

# library untuk Random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
# path dataset
data_path = r'/content/drive/MyDrive/Text Dataset/IMDB Dataset.csv'
dataset = pd.read_csv(data_path, header=0, sep = ',')

# splitting data
train = dataset[0:39999]
test = dataset[40000:49999]

**Preprocessing Dataset**

In [ ]:
# proprocessing data teks
def review_wordlist(review, remove_stopwords=False):
    
    # hapus tag html
    review_text = BeautifulSoup(review).get_text()
    
    # hapus simbol
    review_text = re.sub("[^a-zA-Z]"," ",review_text)
    
    # konversi ke huruf kecil dan dipisah perkata
    words = review_text.lower().split()

    # menghapus stopwords
    if remove_stopwords:
        stops = set(stopwords.words("english"))     
        words = [w for w in words if not w in stops]
    
    return(words)

In [ ]:
# download file punctuation and stpwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# word2vec expects a list of lists.
# Using punkt tokenizer for better splitting of a paragraph into sentences.

import nltk.data
#nltk.download('popular')

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [ ]:
# Memisah review per kalimat
def review_sentences(review, tokenizer, remove_stopwords=False):
    
    # melakukan tokenize dengan nltk tokenizer
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []

    # mengisi array sentences dengan masing - masing review
    for raw_sentence in raw_sentences:
        if len(raw_sentence)>0:
            sentences.append(review_wordlist(raw_sentence,\
                                            remove_stopwords))

    # list dari list
    return sentences

In [ ]:
sentences = []
print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_sentences(review, tokenizer)

Parsing sentences from training set


/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'..'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.happierabroad.com" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


In [ ]:
# Importing the built-in logging module
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
print("List dari lists. Cek tipe data : ", type(sentences), " of ", type(sentences[0]))
print(sentences[10])
print(len(sentences))

List dari lists. Cek tipe data :  <class 'list'>  of  <class 'list'>
['a', 'wonderful', 'little', 'production']
428144


**SEARCH BEST COMBINATION PARAMETER**

In [ ]:
best_score = []
best_parameter = []

In [ ]:
# parameter yang akan dioptimasi
parm_dict = {
    'workers' : (4,),
    'size' : (300, 350, 400, 450, 500),
    'min_count' : (40,),
    'window' : (10,)
}

# melakukan x, dengan metode y, untuk z 
# optimization of feature dimension on word embedding based on vector size using grid search for sentimen analysis

def cust_param_search(parm_dict):

    score_best, parm_best = 0,()
    workers, size, min_count, window = [tup for k,tup in parm_dict.items()] # Individual parm tuples
    parm_combo = list(itertools.product(workers, size, min_count, window)) # Create all combinations
    
    print('\n==============================================================')
    print('PARAMETER')
    print('==============================================================')
    print(f'Total kombinasi parameter   : {len(parm_combo)}')
    print(f'Semua kombinasi parameter   : {parm_combo}')

    # Grid search
    i = 1
    for parms in parm_combo:

        print('\n==============================================================')
        print(f'{i}.\t Parameter : {parms}')
        print('==============================================================')
        
        w, s, m, wi = parms
        
        # =================================================================================
        # word2vec mulai disini
        # =================================================================================
        
        # word2vec model
        model = word2vec.Word2Vec(workers = w, 
                              size = s, 
                              min_count = m, 
                              window = wi)
        model.build_vocab(sentences)

        # training model word2vec (CBOW, karena parameter 'sg' menggunakan nilai default)
        print('\n==============================================================')
        print("Training model Word2Vec ...")
        print('==============================================================')
        model.train(sentences = sentences, total_examples = len(sentences), epochs = 5)

        # Function to average all word vectors in a paragraph
        def featureVecMethod(words, model, num_features):
            # Pre-initialising empty numpy array for speed
            featureVec = np.zeros(s,dtype="float32")
            nwords = 0
            
            #Converting Index2Word which is a list to a set for better speed in the execution.
            index2word_set = set(model.wv.index2word)
            
            for word in  words:
                if word in index2word_set:
                    nwords = nwords + 1
                    featureVec = np.add(featureVec,model[word])
            
            # Dividing the result by number of words to get average
            featureVec = np.divide(featureVec, nwords)
            return featureVec

        # Function for calculating the average feature vector
        def getAvgFeatureVecs(reviews, model, num_features):
            counter = 0
            reviewFeatureVecs = np.zeros((len(reviews),s),dtype="float32")
            for review in reviews:
                # Printing a status message every 1000th review
                if counter%1000 == 0:
                    print("Review %d of %d"%(counter,len(reviews)))
                    
                reviewFeatureVecs[counter] = featureVecMethod(review, model, s)
                counter = counter+1
                
            return reviewFeatureVecs

        # Calculating average feature vector (mendapatkan vektor training set)
        clean_train_reviews = []
        for review in train['review']:
            clean_train_reviews.append(review_wordlist(review, remove_stopwords=True))
            
        trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, s)

        # Calculating average feature vactors (mendapatkan vektor test set)     
        clean_test_reviews = []
        for review in test["review"]:
            clean_test_reviews.append(review_wordlist(review,remove_stopwords=True))
            
        testDataVecs = getAvgFeatureVecs(clean_test_reviews, model, s)

        # tujuan masing-masing model word2vec, untuk mendapatkan 'trainDataVecs' dan 'testDataVecs' 
        # untuk diolah lebih lanjut di classifier
        
        # =================================================================================
        # word2vec berakhir disini
        # =================================================================================

        print('\n==============================================================')
        print('Sentiment Analysis Process ...')
        print('==============================================================')
        # Memanggil classifier
        score, report = classification_model(trainDataVecs, testDataVecs, train["sentiment"], test["sentiment"])

        # temporary print
        print('\n==============================================================')
        print(f'Hasil Test Parameter ke - {i}')
        print(f'Parameter  => {parms}')
        print(f'Accuracy   => {score}')
        print(f'Classification Report => \n {report}')
        print('==============================================================')

        best_score.append(score)
        best_parameter.append(parms)
        
        if score > score_best:
            score_best = score
            parm_best = parms
        
        i = i + 1
    
    print('\n==============================================================')
    print(f'Best Parameter  => {parm_best}')
    print(f'Accuracy   => {score_best}')
    print('==============================================================')

In [ ]:
def classification_model(train_feature_vec, test_feature_vec, train_sentiment, test_sentiment):

    # Model/classifier yang dipakai bebas
    # Fit random forest classifier ke data training
    forest = RandomForestClassifier(n_estimators = 100)    
    forest = forest.fit(train_feature_vec, train_sentiment) 

    # Prediksi nilai sentiment untuk test data 
    predicted = forest.predict(test_feature_vec)

    # akurasi
    accuracy = accuracy_score(test_sentiment, predicted)
    report = classification_report(test_sentiment, predicted, digits = 5)
    
    return accuracy, report

In [ ]:
# test drive ma men (estimasi running : 17 menit)
cust_param_search(parm_dict)

2020-12-22 15:08:26,791 : INFO : collecting all words and their counts
2020-12-22 15:08:26,792 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-12-22 15:08:26,851 : INFO : PROGRESS: at sentence #10000, processed 224307 words, keeping 17319 word types
2020-12-22 15:08:26,908 : INFO : PROGRESS: at sentence #20000, processed 434605 words, keeping 24175 word types
2020-12-22 15:08:26,967 : INFO : PROGRESS: at sentence #30000, processed 655533 words, keeping 29419 word types



PARAMETER
Total kombinasi parameter   : 7
Semua kombinasi parameter   : [(4, 300, 40, 10), (4, 350, 40, 10), (4, 400, 40, 10), (4, 450, 40, 10), (4, 500, 40, 10), (4, 550, 40, 10), (4, 600, 40, 10)]

1.	 Parameter : (4, 300, 40, 10)


2020-12-22 15:08:27,029 : INFO : PROGRESS: at sentence #40000, processed 877306 words, keeping 33897 word types
2020-12-22 15:08:27,092 : INFO : PROGRESS: at sentence #50000, processed 1100000 words, keeping 37667 word types
2020-12-22 15:08:27,151 : INFO : PROGRESS: at sentence #60000, processed 1324580 words, keeping 40923 word types
2020-12-22 15:08:27,209 : INFO : PROGRESS: at sentence #70000, processed 1543432 words, keeping 43631 word types
2020-12-22 15:08:27,268 : INFO : PROGRESS: at sentence #80000, processed 1757050 words, keeping 46072 word types
2020-12-22 15:08:27,330 : INFO : PROGRESS: at sentence #90000, processed 1976526 words, keeping 48547 word types
2020-12-22 15:08:27,398 : INFO : PROGRESS: at sentence #100000, processed 2199610 words, keeping 50850 word types
2020-12-22 15:08:27,473 : INFO : PROGRESS: at sentence #110000, processed 2412028 words, keeping 52887 word types
2020-12-22 15:08:27,538 : INFO : PROGRESS: at sentence #120000, processed 2631483 words, keepin


Training model Word2Vec ...


2020-12-22 15:08:33,103 : INFO : EPOCH 1 - PROGRESS: at 5.24% examples, 332533 words/s, in_qsize 7, out_qsize 1
2020-12-22 15:08:34,134 : INFO : EPOCH 1 - PROGRESS: at 10.73% examples, 342093 words/s, in_qsize 7, out_qsize 1
2020-12-22 15:08:35,146 : INFO : EPOCH 1 - PROGRESS: at 16.20% examples, 347376 words/s, in_qsize 7, out_qsize 0
2020-12-22 15:08:36,164 : INFO : EPOCH 1 - PROGRESS: at 21.67% examples, 347892 words/s, in_qsize 6, out_qsize 1
2020-12-22 15:08:37,175 : INFO : EPOCH 1 - PROGRESS: at 27.22% examples, 349822 words/s, in_qsize 6, out_qsize 1
2020-12-22 15:08:38,208 : INFO : EPOCH 1 - PROGRESS: at 32.69% examples, 349918 words/s, in_qsize 4, out_qsize 3
2020-12-22 15:08:39,209 : INFO : EPOCH 1 - PROGRESS: at 38.36% examples, 351776 words/s, in_qsize 7, out_qsize 0
2020-12-22 15:08:40,247 : INFO : EPOCH 1 - PROGRESS: at 43.75% examples, 350597 words/s, in_qsize 5, out_qsize 2
2020-12-22 15:08:41,271 : INFO : EPOCH 1 - PROGRESS: at 49.33% examples, 351042 words/s, in_qsize

...


Training model Word2Vec ...


2020-12-22 15:42:30,467 : INFO : EPOCH 1 - PROGRESS: at 3.52% examples, 227832 words/s, in_qsize 8, out_qsize 1
2020-12-22 15:42:31,485 : INFO : EPOCH 1 - PROGRESS: at 7.46% examples, 240816 words/s, in_qsize 7, out_qsize 0
2020-12-22 15:42:32,538 : INFO : EPOCH 1 - PROGRESS: at 11.05% examples, 235478 words/s, in_qsize 7, out_qsize 0
2020-12-22 15:42:33,559 : INFO : EPOCH 1 - PROGRESS: at 14.55% examples, 234551 words/s, in_qsize 7, out_qsize 0
2020-12-22 15:42:34,570 : INFO : EPOCH 1 - PROGRESS: at 18.46% examples, 237338 words/s, in_qsize 6, out_qsize 1
2020-12-22 15:42:35,656 : INFO : EPOCH 1 - PROGRESS: at 22.49% examples, 238502 words/s, in_qsize 7, out_qsize 0
2020-12-22 15:42:36,704 : INFO : EPOCH 1 - PROGRESS: at 26.57% examples, 240466 words/s, in_qsize 7, out_qsize 0
2020-12-22 15:42:37,707 : INFO : EPOCH 1 - PROGRESS: at 30.38% examples, 241630 words/s, in_qsize 7, out_qsize 0
2020-12-22 15:42:38,708 : INFO : EPOCH 1 - PROGRESS: at 34.09% examples, 241857 words/s, in_qsize 

Review 0 of 39999


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Review 1000 of 39999
Review 2000 of 39999
Review 3000 of 39999
Review 4000 of 39999
Review 5000 of 39999
Review 6000 of 39999
Review 7000 of 39999
Review 8000 of 39999
Review 9000 of 39999
Review 10000 of 39999
Review 11000 of 39999
Review 12000 of 39999
Review 13000 of 39999
Review 14000 of 39999
Review 15000 of 39999
Review 16000 of 39999
Review 17000 of 39999
Review 18000 of 39999
Review 19000 of 39999
Review 20000 of 39999
Review 21000 of 39999
Review 22000 of 39999
Review 23000 of 39999
Review 24000 of 39999
Review 25000 of 39999
Review 26000 of 39999
Review 27000 of 39999
Review 28000 of 39999
Review 29000 of 39999
Review 30000 of 39999
Review 31000 of 39999
Review 32000 of 39999
Review 33000 of 39999
Review 34000 of 39999
Review 35000 of 39999
Review 36000 of 39999
Review 37000 of 39999
Review 38000 of 39999
Review 39000 of 39999
Review 0 of 9999
Review 1000 of 9999
Review 2000 of 9999
Review 3000 of 9999
Review 4000 of 9999
Review 5000 of 9999
Review 6000 of 9999
Review 7000 of

In [ ]:
print(best_score)
print(best_parameter)

[0.8283828382838284, 0.8264826482648265, 0.8294829482948295, 0.831983198319832, 0.8310831083108311, 0.832083208320832, 0.8323832383238324]
[(4, 300, 40, 10), (4, 350, 40, 10), (4, 400, 40, 10), (4, 450, 40, 10), (4, 500, 40, 10), (4, 550, 40, 10), (4, 600, 40, 10)]
